# AudioEngine Initialization with ASIO Devices
This notebook examines and implements AudioEngine initialization and configuration using ASIO devices. We'll walk through detecting ASIO devices, loading configurations from JSON, and setting up the proper audio processing pipeline.

## Import Required Libraries
Import necessary libraries, including JSON handling, ASIO SDK bindings, and any custom AudioEngine and AsioManager modules.

In [ ]:
import json
import sys
import os
import numpy as np
from pathlib import Path

# Import custom modules
sys.path.append('../')  # Adjust if needed to locate the modules
from audio_engine import AudioEngine
from asio_manager import AsioManager
from audio_node import AudioNode

# If using third-party ASIO bindings
try:
    import pyasio  # This is placeholder - use your actual ASIO binding
except ImportError:
    print("Warning: pyasio module not found. You'll need proper ASIO bindings.")

## Initialize AudioEngine with JSON Configuration
Load a JSON file containing the ASIO device name, sampling rate, and buffer size. Pass this configuration to the AudioEngine object during initialization.

In [ ]:
# Define a sample configuration JSON
config_json = {
    "audio": {
        "device": "ASIO4ALL v2",  # Example ASIO device
        "sampling_rate": 48000,
        "buffer_size": 512,
        "channels": {
            "input": 2,
            "output": 2
        }
    }
}

# Save configuration to file
config_path = Path("./audio_config.json")
with open(config_path, 'w') as f:
    json.dump(config_json, f, indent=4)

print(f"Configuration saved to {config_path.absolute()}")

# Load configuration from file
with open(config_path, 'r') as f:
    loaded_config = json.load(f)

# Initialize AudioEngine with configuration
audio_engine = AudioEngine()
initialization_result = audio_engine.initialize(loaded_config["audio"])

print(f"AudioEngine initialized: {initialization_result}")

## Detect ASIO Device Using AsioManager
Use the AsioManager object to detect the specified ASIO device and validate its availability.

In [ ]:
# Create AsioManager instance
asio_manager = AsioManager()

# Get list of available ASIO devices
available_devices = asio_manager.get_device_list()
print("Available ASIO devices:")
for idx, device in enumerate(available_devices):
    print(f"{idx + 1}. {device}")

# Check if our configured device is available
target_device = loaded_config["audio"]["device"]
if target_device in available_devices:
    print(f"\nTarget device '{target_device}' is available.")
    # Select the device
    selection_result = asio_manager.select_device(target_device)
    print(f"Device selection result: {selection_result}")
else:
    print(f"\nWARNING: Target device '{target_device}' is not available!")
    if available_devices:
        # If target not found but others exist, select the first one
        alt_device = available_devices[0]
        print(f"Selecting alternative device: {alt_device}")
        selection_result = asio_manager.select_device(alt_device)
        print(f"Device selection result: {selection_result}")

## Retrieve ASIO Device Configuration
Query the ASIO device for its input/output channels, actual sampling rate, and buffer size using the ASIO SDK.

In [ ]:
# Query device capabilities after selection
if hasattr(asio_manager, 'get_device_info') and asio_manager.is_device_selected():
    device_info = asio_manager.get_device_info()

    print("\nASIO Device Information:")
    print(f"Device Name: {device_info['name']}")
    print(f"Driver Version: {device_info['driver_version']}")
    print(f"Available Input Channels: {device_info['input_channels']}")
    print(f"Available Output Channels: {device_info['output_channels']}")
    print(f"Minimum Buffer Size: {device_info['min_buffer_size']}")
    print(f"Maximum Buffer Size: {device_info['max_buffer_size']}")
    print(f"Preferred Buffer Size: {device_info['preferred_buffer_size']}")
    print(f"Supported Sample Rates: {device_info['supported_sample_rates']}")

    # Check if our configured sample rate and buffer size are supported
    target_sample_rate = loaded_config["audio"]["sampling_rate"]
    target_buffer_size = loaded_config["audio"]["buffer_size"]

    if target_sample_rate in device_info['supported_sample_rates']:
        print(f"\nTarget sample rate {target_sample_rate}Hz is supported.")
    else:
        print(f"\nWARNING: Target sample rate {target_sample_rate}Hz is NOT supported!")
        print(f"Consider using one of: {device_info['supported_sample_rates']}")

    if (device_info['min_buffer_size'] <= target_buffer_size <= device_info['max_buffer_size']):
        print(f"Target buffer size {target_buffer_size} is supported.")
    else:
        print(f"WARNING: Target buffer size {target_buffer_size} is outside the allowed range!")
        print(f"Allowed range: {device_info['min_buffer_size']} - {device_info['max_buffer_size']}")
        print(f"Consider using preferred size: {device_info['preferred_buffer_size']}")
else:
    print("Cannot query device information - no device selected or method not available")

## Create AudioNode Matrix
Use the retrieved ASIO device configuration to create an initial AudioNode matrix for sound playback.

In [ ]:
# Create AudioNode matrix based on retrieved configuration
def create_audio_node_matrix(input_channels, output_channels):
    """Create a matrix of AudioNode objects based on channel configuration"""
    input_nodes = []
    output_nodes = []

    # Create input nodes
    for i in range(input_channels):
        node = AudioNode(f"input_{i}", node_type="input")
        input_nodes.append(node)

    # Create output nodes
    for i in range(output_channels):
        node = AudioNode(f"output_{i}", node_type="output")
        output_nodes.append(node)

    # Create a simple processing node (e.g., gain)
    gain_node = AudioNode("gain_node", node_type="processor")

    # Connect nodes (simple stereo config example)
    if input_channels >= 2 and output_channels >= 2:
        # Connect input 0,1 to gain node
        input_nodes[0].connect_to(gain_node)
        input_nodes[1].connect_to(gain_node)

        # Connect gain node to outputs 0,1
        gain_node.connect_to(output_nodes[0])
        gain_node.connect_to(output_nodes[1])

    return {
        "inputs": input_nodes,
        "outputs": output_nodes,
        "processors": [gain_node]
    }

# Get the actual configuration to use
if 'device_info' in locals():
    # We have retrieved device info
    actual_inputs = min(device_info['input_channels'], loaded_config["audio"]["channels"]["input"])
    actual_outputs = min(device_info['output_channels'], loaded_config["audio"]["channels"]["output"])
else:
    # Fallback to config values
    actual_inputs = loaded_config["audio"]["channels"]["input"]
    actual_outputs = loaded_config["audio"]["channels"]["output"]

# Create the node matrix
node_matrix = create_audio_node_matrix(actual_inputs, actual_outputs)

print(f"Created AudioNode matrix with:")
print(f"- {len(node_matrix['inputs'])} input nodes")
print(f"- {len(node_matrix['outputs'])} output nodes")
print(f"- {len(node_matrix['processors'])} processing nodes")

# Register nodes with AudioEngine
if hasattr(audio_engine, "register_nodes"):
    for node_type, nodes in node_matrix.items():
        for node in nodes:
            audio_engine.register_node(node)
    print("All nodes registered with AudioEngine")

## Update Framework Code for Hardware and Software Configuration
Modify the framework code to implement the required hardware and software configuration for the AudioEngine and ASIO device integration.

In [ ]:
# Define a function to update the ASIO callback handlers in our framework
def update_asio_framework_code():
    """
    Update or generate the framework code to handle ASIO callbacks and configuration
    """
    # Example framework code that might need to be generated/updated
    asio_callback_code = """
class AsioCallbacks:
    def __init__(self, audio_engine):
        self.audio_engine = audio_engine

    def buffer_switch(self, input_buffers, output_buffers, buffer_size, sample_time):
        # Process audio: get input from device, process through our nodes, send to output
        # This is where our AudioEngine processing happens
        self.audio_engine.process_audio(input_buffers, output_buffers, buffer_size)

    def sample_rate_changed(self, sample_rate):
        # Handle sample rate changes from the ASIO device
        self.audio_engine.update_sample_rate(sample_rate)

    def reset_request(self):
        # Handle ASIO reset requests
        return self.audio_engine.reset()

    def buffer_size_changed(self, new_size):
        # Handle buffer size changes
        self.audio_engine.update_buffer_size(new_size)
"""

    # Integration code for connecting our framework to ASIO
    integration_code = """
def initialize_audio_engine_with_asio(config):
    # Create instances
    engine = AudioEngine()
    asio_mgr = AsioManager()

    # Configure ASIO
    asio_mgr.select_device(config["device"])
    asio_mgr.set_sample_rate(config["sampling_rate"])
    asio_mgr.set_buffer_size(config["buffer_size"])

    # Connect engine to ASIO
    callbacks = AsioCallbacks(engine)
    asio_mgr.set_callbacks(callbacks)

    # Start audio processing
    engine.initialize(config)
    asio_mgr.start()

    return engine, asio_mgr
"""

    # Print the example code
    print("Example ASIO callback handler code:")
    print(asio_callback_code)
    print("\nExample integration code:")
    print(integration_code)

    # In a real scenario, we might write this to actual implementation files
    # with open("../audio_engine/asio_callbacks.py", "w") as f:
    #     f.write(asio_callback_code)
    # with open("../audio_engine/asio_integration.py", "w") as f:
    #     f.write(integration_code)

    return True

# Update the framework code
update_result = update_asio_framework_code()
print(f"\nFramework code update successful: {update_result}")

# Final summary
print("\n=== AudioEngine ASIO Configuration Summary ===")
print(f"Target ASIO Device: {loaded_config['audio']['device']}")
print(f"Sample Rate: {loaded_config['audio']['sampling_rate']} Hz")
print(f"Buffer Size: {loaded_config['audio']['buffer_size']} samples")
print(f"Input Channels: {actual_inputs}")
print(f"Output Channels: {actual_outputs}")
print(f"Audio Node Matrix: {len(node_matrix['inputs'])} inputs, {len(node_matrix['outputs'])} outputs, {len(node_matrix['processors'])} processors")
print("================================================")